In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
llm_config = {"model": "gpt-3.5-turbo"}

In [7]:
task = '''
        Write a concise but engaging blogpost about
        Gen AI. Make sure the blogpost is
       within 10 words.
       '''

In [8]:
import autogen

In [9]:
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [10]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [11]:
print(reply)

Title: "Gen AI: The Future of Innovation in Technology"

Unleash Gen AI: A game-changer revolutionizing tech innovation.


### **Adding Reflection**

In [12]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [13]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
        Gen AI. Make sure the blogpost is
       within 10 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Gen AI: A Bright Future Beyond Imagination"

Blogpost:
Gen AI: The dawn of endless possibilities in technology.

--------------------------------------------------------------------------------
Critic (to Writer):

Your blogpost is succinct and captures the essence of the topic effectively. It creates curiosity and intrigue about Gen AI. To enhance the engagement of readers, consider expanding on the potential applications and impacts of Gen AI in a few more sentences. This will provide readers with more insightful information and make the blogpost more compelling. Additionally, including real-world examples or statistics can add credibility to your content. Keep up the good work!

--------------------------------------------------

## **Nested chat**

In [31]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO-Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [32]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal-Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [33]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics-Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [34]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

# **Start Nested Chat**

In [35]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer,
    "message": reflection_message, 
    "summary_method": "reflection_with_llm",
    "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]

In [36]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [37]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
        Gen AI. Make sure the blogpost is
       within 10 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unveiling Gen AI: The Future of Artificial Intelligence"

In this blog post, we'll explore the exciting realm of Gen AI.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to SEO-Reviewer):

Review the following content. 
            

 Title: "Unveiling Gen AI: The Future of Artificial Intelligence"

In this blog post, we'll explore the exciting realm of Gen AI.

--------------------------------------------------------------------------------
SEO-Reviewer (to Critic):

As an SEO reviewer, here are my suggest